# Analyzing the NYC Taxi dataset using RAPIDS

This notebook uses [RAPIDS](https://rapids.ai/) for accelerating analytics using GPU. Specifically we use the `ml.g4dn.xlarge` instance type that has 4 vCPUs, 16 GB RAM and 1 GPU. We run the same analytics tasks that we have run earlier on the NYC Taxi dataset that we have done previously with Spark, Dask and DuckDB. 

This notebook does require installing `%conda create -n rapids-22.08 -c rapidsai -c nvidia -c conda-forge rapids=22.08 python=3.9 cudatoolkit=11.5 dask-sql ipykernel -y` but it is best to package this conda environment in a container and attach that container to SageMaker Studio. This notebook assumes that the kernel being used is this custom kernel which has the `rapids` and `cudatoolkit` pre-installed. The details of this process are beyond the scope of this notebook [README.md](./README.md) for more details.

In [2]:
!pip install s3fs==2022.11.0 gputil==1.4.0

In [3]:
import os
import cudf
import GPUtil
import dask_cudf
import cupy as cp
import pandas as pd


In [4]:
cudf_df = cudf.read_parquet('s3://bigdatateaching/nyctaxi-yellow-tripdata/2021/*.parquet')

In [5]:
cudf_df.shape

(1369769, 19)

In [6]:
cudf_df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2021-01-01 00:30:10,2021-01-01 00:36:12,1.0,2.10,1.0,N,142,43,2,8.0,3.0,0.5,0.00,0.0,0.3,11.80,2.5,<NA>
1,1,2021-01-01 00:51:20,2021-01-01 00:52:19,1.0,0.20,1.0,N,238,151,2,3.0,0.5,0.5,0.00,0.0,0.3,4.30,0.0,<NA>
2,1,2021-01-01 00:43:30,2021-01-01 01:11:06,1.0,14.70,1.0,N,132,165,1,42.0,0.5,0.5,8.65,0.0,0.3,51.95,0.0,<NA>
3,1,2021-01-01 00:15:48,2021-01-01 00:31:01,0.0,10.60,1.0,N,138,132,1,29.0,0.5,0.5,6.05,0.0,0.3,36.35,0.0,<NA>
4,2,2021-01-01 00:31:49,2021-01-01 00:48:21,1.0,4.94,1.0,N,68,33,1,16.5,0.5,0.5,4.06,0.0,0.3,24.36,2.5,<NA>


In [7]:
cudf_df.tpep_pickup_datetime.dtype

dtype('<M8[us]')

In [8]:
cudf_df['day'] = cudf.to_datetime(cudf_df['tpep_pickup_datetime']).dt.day
cudf_df['month'] = cudf.to_datetime(cudf_df['tpep_pickup_datetime']).dt.month
cudf_df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,day,month
0,1,2021-01-01 00:30:10,2021-01-01 00:36:12,1.0,2.10,1.0,N,142,43,2,...,3.00,0.5,0.00,0.00,0.3,11.80,2.5,<NA>,1,1
1,1,2021-01-01 00:51:20,2021-01-01 00:52:19,1.0,0.20,1.0,N,238,151,2,...,0.50,0.5,0.00,0.00,0.3,4.30,0.0,<NA>,1,1
2,1,2021-01-01 00:43:30,2021-01-01 01:11:06,1.0,14.70,1.0,N,132,165,1,...,0.50,0.5,8.65,0.00,0.3,51.95,0.0,<NA>,1,1
3,1,2021-01-01 00:15:48,2021-01-01 00:31:01,0.0,10.60,1.0,N,138,132,1,...,0.50,0.5,6.05,0.00,0.3,36.35,0.0,<NA>,1,1
4,2,2021-01-01 00:31:49,2021-01-01 00:48:21,1.0,4.94,1.0,N,68,33,1,...,0.50,0.5,4.06,0.00,0.3,24.36,2.5,<NA>,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1369764,2,2021-01-31 23:03:00,2021-01-31 23:33:00,<NA>,8.89,<NA>,<NA>,229,181,0,...,0.00,0.5,7.46,0.00,0.3,38.54,<NA>,<NA>,31,1
1369765,2,2021-01-31 23:29:00,2021-01-31 23:51:00,<NA>,7.43,<NA>,<NA>,41,70,0,...,0.00,0.5,0.00,6.12,0.3,39.50,<NA>,<NA>,31,1
1369766,2,2021-01-31 23:25:00,2021-01-31 23:38:00,<NA>,6.26,<NA>,<NA>,74,137,0,...,0.00,0.5,3.90,0.00,0.3,24.05,<NA>,<NA>,31,1
1369767,6,2021-01-31 23:01:06,2021-02-01 00:02:03,<NA>,19.70,<NA>,<NA>,265,188,0,...,0.00,0.5,0.00,0.00,0.3,54.48,<NA>,<NA>,31,1


In [9]:
%time
cudf_df.groupby(['month', 'day']).agg({'trip_distance': "mean",
                                       'fare_amount': "mean",
                                       'mta_tax': 'mean',
                                       'tip_amount': 'mean',
                                       'total_amount': 'mean',
                                       'congestion_surcharge': 'mean'})

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.48 µs


trip_distance  fare_amount   mta_tax  tip_amount  total_amount  \
month day                                                                   
1     2         3.441793    13.502239  0.491314    2.145086     18.954313   
      23        2.764356    11.414721  0.492569    1.860940     16.605046   
      17        3.260376    12.720475  0.492981    2.061331     18.032450   
      20        5.550783    11.506614  0.493568    1.806717     16.913929   
      3         9.749994    14.789198  0.489638    2.316048     20.452838   
      6         4.861730    11.989868  0.493319    1.912365     17.352783   
      18        3.056818    12.305442  0.493071    2.014083     17.612088   
      9         5.164707    12.189154  0.492308    1.995121     17.436535   
      21        3.231620    11.967786  0.494247    1.852851     17.441953   
      24        3.242845    12.528322  0.492266    1.969346     17.856120   
      13        5.874701    11.665974  0.494065    1.908853     17.017836   
      26        5.300479    11.493175  0.493517    1.778217     16.848290   
      4         3.889088    13.051403  0.492872    2.009752     18.533799   
      31        6.683928    12.046356  0.492432    1.921767     17.291320   
2     1         1.266667     6.833333  0.500000    0.846667     10.980000   
1     10        5.158755    13.214621  0.492011    2.138281     18.635103   
      8         3.792919    11.861371  0.493135    1.929520     17.259475   
      22        2.696882    11.729058  0.493359    1.834794     17.189448   
2     22        0.720000     4.500000  0.500000    1.760000     10.560000   
1     19        3.783150    11.927354  0.493920    1.858116     17.386038   
      25        2.799093    11.949387  0.493115    1.776973     17.313478   
      15        2.776716    12.031660  0.493861    1.949246     17.438616   
      16        9.634065    12.034402  0.492679    1.982170     17.272636   
12    31        4.531176    15.000000  0.470588    1.381176     19.975294   
1     30        2.789168    11.568800  0.492799    1.870083     16.762451   
      29        9.794840    11.569862  0.493398    1.842056     17.027418   
      28        8.427085    11.565915  0.493604    1.816441     16.992320   
      1         3.580558    13.529165  0.487435    1.986472     18.812830   
      14        4.232506    12.039560  0.493610    1.939516     17.439517   
      5         4.047962    12.347000  0.493174    1.933841     17.716886   
      7         3.885199    12.213134  0.493339    1.930224     17.617980   
      11        5.784297    12.090008  0.493383    1.915442     17.431219   
      27        2.630664    11.580591  0.493638    1.801726     16.986395   
      12        2.764806    11.951595  0.493039    1.926372     17.318471   

           congestion_surcharge  
month day                        
1     2                2.156934  
      23               2.257270  
      17               2.205055  
      20               2.276005  
      3                2.080468  
      6                2.231545  
      18               2.232386  
      9                2.207145  
      21               2.287558  
      24               2.195916  
      13               2.280199  
      26               2.282151  
      4                2.182669  
      31               2.211831  
2     1                2.500000  
1     10               2.154927  
      8                2.232020  
      22               2.276595  
2     22               2.500000  
1     19               2.265042  
      25               2.254727  
      15               2.252356  
      16               2.226106  
12    31               2.205882  
1     30               2.242061  
      29               2.274536  
      28               2.288171  
      1                2.096757  
      14               2.266379  
      5                2.214747  
      7                2.235868  
      11               2.227675  
      27               2.288136  
      12               2.249007

In [10]:
%%time
for i in range(1000):
    cudf_df.groupby(['month', 'day']).agg({'trip_distance': "mean",
                                           'fare_amount': "mean",
                                           'mta_tax': 'mean',
                                           'tip_amount': 'mean',
                                           'total_amount': 'mean',
                                           'congestion_surcharge': 'mean'})
    if i % 100 == 0:
        print(GPUtil.showUtilization())

| ID | GPU | MEM |
------------------
|  0 |  1% |  4% |
None
| ID | GPU | MEM |
------------------
|  0 | 61% |  4% |
None
| ID | GPU | MEM |
------------------
|  0 | 62% |  4% |
None
| ID | GPU | MEM |
------------------
|  0 | 62% |  4% |
None
| ID | GPU | MEM |
------------------
|  0 | 62% |  4% |
None
| ID | GPU | MEM |
------------------
|  0 | 61% |  4% |
None
| ID | GPU | MEM |
------------------
|  0 | 61% |  4% |
None
| ID | GPU | MEM |
------------------
|  0 | 61% |  4% |
None
| ID | GPU | MEM |
------------------
|  0 | 62% |  4% |
None
| ID | GPU | MEM |
------------------
|  0 | 62% |  4% |
None
CPU times: user 6.81 s, sys: 7.08 s, total: 13.9 s
Wall time: 14.1 s
